<center><img src="images/header.png"></center>

<h1><center>Лекция №6: Введение в анализ социальных сетей</center></h1>
<hr>
<h1><center>Домашняя работа</center></h1>

Данное ДЗ является довольно творческим и оно требует довольно много самостоятельной работы, так что не стоит с ним мешкать =)

Вам предлагается выбрать сообщество (или сообщества) в сети ВКонтакте и проанализировать его (их) контингент. 


**Правила:**
* Отчеты принимаются до **23:59 19 ноября**. Вы должны залить ваш ноутбук в github дать на него ссылку в следующей [форме](https://forms.gle/88d9CfaJN5gPV4bB8).
* Работоспособность будет проверяться через [colab](https://colab.research.google.com) (можете сразу давать ссылку на него). Если ноутбук не будет исполняться (падать по ошибке) то все, что будет написано после ошибки не будет учитываться при формировании оценки. Для самопроверки, выполните аналог restart kernel and run all cells в colab
* Вы должны включить код, с помощью которого вы общались с API вконтакте, но для экономии времени, заранее предусмотреть возможность загрузки уже собранных данных (например, как это было реализовано в ноутбуках по кластеризации через `wget` и dropbox или raw данными в github)

**Пожелания:**
* Выбирайте для анализа небольшие сообщества (до 5к), желательно чтобы вы сами в них состояли - это ускорит вычисления и упростит интерпретацию
* Отнеситесь к этой работе как к исследованию - больше графиков и комментариев!
* Красивые визуализации лучше делать через gephi!

# Загрузка данных (2 балла)

Изучите следующие материалы
* [Знакомство с API](https://vk.com/dev/first_guide)
* [Туториал](https://nbviewer.jupyter.org/github/allatambov/Py-programming-3/blob/master/15-06/lect-vk-api.ipynb) по выгрузке данных с помощью VKAPI
* Про метод execute - [туть](https://vk.com/dev/execute) и [туть](http://www.cyberforum.ru/python-web/thread1730446.html)

Выберите сообщество и загрузите
* Идентификаторы всех [участников сообщества](https://vk.com/dev/groups.getMembers)
* [Списки друзей](https://vk.com/dev/friends.get) участников и их признаки (пол, возраст, университет и все, что посчитаете нужным - нам потом группировать по этим атрибутам)
* [Пересечение](https://vk.com/dev/friends.getMutual) между списками друзей

Таким образом вы должны получить замкнутую группу аккаунтов с их друзьями и связями между ними.

Далее - соберите их этого граф в networkX и сохраните в graphml или gml формате

**Замечания:**
* Вконтакте может ограничить вам скорость запросов, поэтому иногда имеет смысл ставить небольшую паузу (`time.sleep(1)`) между запросами и сохранять предварительные результаты
* Код выгрузки может быть написан как с помощью модуля `vk` так и просто с помощью requests
* Некоторые пользователи могли сделать свой аккаунт приватным, что может вызвать ошибку в методе `getMutual`. Чтобы этого избежать придется проверять каждого на приватность с помощью `users.get` и аттрибута `is_closed`


In [42]:
### Здесь должен быть код загрузки из VK
### Его можно закомментировать
#!pip install vk
import vk
token = '270dd565c8ad87d7faf557a72c53e6913899f4d8eac190c3bed75aa35c8d62556ffaac360b8a5c7729ff1'
session = vk.Session(access_token=token)
api = vk.API(session=session)

In [46]:
group = 'dastanzionno'

In [47]:
all_members = api.groups.getMembers(group_id=group, v = 5.73)
members = all_members['items']

In [48]:
import time

In [78]:
members_inform = []
for i in members:
    time.sleep(1)
    members_inform.append(api.users.get(user_ids = i, fields='sex, city, education, universities', v = 5.89))

In [ ]:
members_friends = []
for i in members:
    time.sleep(1)
    if not api.users.get(user_ids = i, v = 5.89)[0]['is_closed']:
        members_friends.append(api.friends.get(user_id=i, fields='sex, city, education, universities', v = 5.89))

In [65]:
import networkx as nx
import json
### Здесь должен быть код преобразования в граф и сохранение в файл
### Его тоже можно закомментировать

In [94]:
g = nx.Graph()
g.graph['title'] = 'hw_networks'
for i in members_inform:
    g.add_node(i[0]['id'])
    if ('is_closed' in i[0] and not i[0]['is_closed']):
        g.node[i[0]['id']]['first_name'] = i[0]['first_name']
        g.node[i[0]['id']]['last_name'] = i[0]['last_name']
        g.node[i[0]['id']]['sex'] = 'man' if (i[0]['sex'] == 2) else 'woman'
        if ('city' in i[0]):
            g.node[i[0]['id']]['city'] = i[0]['city']['title']
        if ('university_name' in i[0]):
            g.node[i[0]['id']]['university'] = i[0]['university_name']

In [ ]:
for i in range(len(members_charact)):
    for j in members_charact[i]:
        g.add_edge(members_inform[i]['id'] , j['id'])
        g.node[j['id']]['sex'] = j['sex']
        g.node[j['id']]['city'] = j['city']['title']
        g.node[j['id']]['education'] = j['education']
        g.node[j['id']]['universities'] = j['universities']

In [31]:
nx.readwrite.gml.write_gml(g, "graph.gml")

{'count': 142,
 'items': [{'first_name': 'Яна',
   'id': 632,
   'last_name': 'Кальней',
   'can_access_closed': True,
   'is_closed': False,
   'sex': 1,
   'city': {'id': 2, 'title': 'Санкт-Петербург'},
   'track_code': 'd0fba2b33EVx4pvRXaNxzZk7B1UQKlEubt1urAeEcpzeWbmpCa2xLiuGqIVdpE6Z-K3RroEoQw'},
  {'first_name': 'Любовь',
   'id': 1401,
   'last_name': 'Каменная',
   'can_access_closed': False,
   'is_closed': True,
   'sex': 1,
   'city': {'id': 2, 'title': 'Санкт-Петербург'},
   'track_code': 'cb7d6116B8wDWq03J1ACb4DrAlCdCDOE65pCzmQ8Z73C-x4Zh_xqp1k5yj9xU1cI7G7NpRt4IeI'},
  {'first_name': 'Дарьяна',
   'id': 1859,
   'last_name': 'Димитрова',
   'can_access_closed': False,
   'is_closed': True,
   'sex': 1,
   'city': {'id': 2, 'title': 'Санкт-Петербург'},
   'track_code': '2afdaf0coqbJgiFZ_jJGU5siMN5p0yzZxie3h67fGaWohH6JbMvPzcOxFArxME4w9Kf_K--jPr8'},
  {'first_name': 'Тимур',
   'id': 5898,
   'last_name': 'Пикуш',
   'can_access_closed': False,
   'is_closed': True,
   'sex': 2,

In [3]:
### Здесь код, который загружает сохраненную сеть и все необходимые данные через wget

# Визуализация (1 балл)

* Откройте полученный граф в gephi
* Выберите наиболее удачный на ваш взгляд алгоритм прорисовки
* Добавьте изменение размера вершины в зависимости от степени вершины
* Вставьте скрин-шот ниже

In [4]:
### Ваши картинки и комментарии ниже

# Центральности вершин (2 балла)

Возможно, перед тем как пойти дальше, стоит немного почитстить граф - с помощью [kcore](https://www.google.com/search?client=firefox-b-d&q=networkx+kcore)

* Найдите наиболее влиятельные вершины с точки зрения degree, closeness и betweenness centrality
* Выделите их на картинке в gephi

In [5]:
### Ваш код, картинки и комментарии ниже

# Сообщества... в сообществе (2 балла)

В этом задании надо примерить 2 алгоритма выявления сообществ и сравнить их результаты

Выберите алгоритм выделения сообществ на графах, например
* label propagation или 
* алгоритм на основе edge betweeness

и алгоритм на основе похожести вершин, например 
* расчитатайте попарную похожесть по jaccard, косинусному расстоянию или корреляцию
* при необходимости переведите "похожесть" в "расстояние" (чем больше значение тем "дальше" объекты друг от друга)
* подайте матрицу попарных расстояний в алгоритм кластеризации, например в DBSCAN

Для обоих вариантов
* Выделите сообщества, опираясь на максимальное значение модулярности для полученного разбиения
* Изобразите граф с вершинами, раскрашенными в цвет своего сообщества в gephi
* Попробуйте проинтерпретировать полученные сообщества, кто эти люди, почему они связаны?
* Какой метод на ваш взгляд сработал лучше?

In [7]:
### Ваш код, картинки и комментарии ниже